## Generate VEX function completions

Finally, verified signatures documentation in Houdini 16.5!
I'll discard all previous work I did in last couple of years
and will rely on the docs.

In [1]:
import sys
import pathlib
import zipfile
import json

from hfs import HFS
sys.path.append(str(HFS / 'houdini' / 'python3.10libs'))
sys.path.append(str(HFS / 'python310' / 'lib' / 'site-packages'))
from bookish.wiki.wikipages import parse_to_root as parse_wikipage
from houdinihelp import vex

In [2]:
def extract_text(node):
    """Evaluate possibly nested 'text' element to string."""
    if isinstance(node, dict):
        return extract_text(node['text'])
    elif isinstance(node, list):
        return ''.join(extract_text(i) for i in node)
    else:
        return node


def usages(node):
    """Recursively extract all 'type' values from Bookish tree."""
    if type(node) is dict:
        if node['type'] == 'usage':
            yield extract_text(node['text'])

        for k in node:
            if type(node[k]) is list:
                for i in node[k]:
                    for j in usages(i):
                        yield j


def all_usages(functions):
    """Convenience loop for many Bookish trees stored in a dict."""
    for f in functions:
        for u in usages(functions[f]):
            yield u

In [3]:
# Open vex.zip containing various helpcards including functions.
# Parse Wiki files into JSON using Bookish wiki parser.
docs = {}

with zipfile.ZipFile(HFS / 'houdini' / 'help' / 'vex.zip') as z:
    for path in z.namelist():
        path = pathlib.PurePath(path)
        if path.parent.stem == 'functions':
            with z.open(path.as_posix()) as f:
                name = path.stem
                markup = f.read().decode()
                docs[name] = parse_wikipage(markup)

print(len(docs))

1098


In [4]:
# Parse function usage examples from docs.
function_usages = [vex.parse_vex(usage) for usage in all_usages(docs)]

print(len(function_usages))
print(function_usages[0])

2293
<sig <type 'int'> <'abs'> (<arg <type 'int'> <'n'> out=False opt=None>)>


In [5]:
# Parse function summary lines from help cards.
function_summaries = {}

for func_name, page in docs.items():
    for elem in page['body']:
        if elem['type'] == 'summary':
            summary_text = extract_text(elem['text'])
            summary_text = ' '.join(summary_text.split()).rstrip('.')
            function_summaries[func_name] = summary_text
            break

print(len(function_summaries))
print(f"point: {function_summaries['point']}")

1035
point: Reads a point attribute value from a geometry


In [6]:
comps = {
    'scope': 'source.vex -string -comment -source.hscript',
    'completions': []
}
used_triggers = set()  # keep track of used completions to avoid duplicates

for usage in function_usages:
    function_name = usage.ident.name

    args = []
    for arg in usage.arglist.args:
        if isinstance(arg, vex.VariadicArgs):
            args.append(arg.string())
        else:
            args.append(arg.ident.string())

    trigger = f"{function_name}({', '.join(args)})"

    # Check if the trigger exists already.
    if trigger in used_triggers:
        continue

    # Completion does not exist. Make a new one.
    used_triggers.add(trigger)

    completion = {'trigger': trigger}

    # Build contents.
    cargs = [f'${{{i}:{arg}}}' for i, arg in enumerate(args, 1)]
    completion['contents'] = f"{function_name}({', '.join(cargs)})"

    # Specify kind.
    completion['kind'] = 'function'

    # Add a summary line, if present.
    if function_name in function_summaries:
        completion['details'] = function_summaries[function_name]

    comps['completions'].append(completion)


print('Generated %d completions.' % len(used_triggers))


# Write completions into a functions.sublime-completions file.
with open('functions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 1771 completions.
